In [1]:
from sklearn.svm import SVC
import pandas as pd
import numpy as np
import os
import glob
import import_ipynb
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from preprocess_data import filter_col, get_table
import pickle

In [2]:
def create_data(participant, mood, frame_to_keep=150, start=150, 
         train_percentage=17/22, base_path=None):
    df = get_table(participant, mood)
    df = filter_col(df)
    df.drop(df.columns[0], axis=1, inplace=True)
    train_index = int((len(df)-start)*train_percentage)
    
    # get 4 features of each frame in 150 consecutive frames (15s)
    # and concat it into a vector
    X = [df[index-frame_to_keep:index].to_numpy().reshape(frame_to_keep*4)
              for index in range(start, len(df))]
    y = np.full((len(df)-start, 1), mood, dtype=int)
    X_train = np.array(X[:train_index])
    X_test = np.array(X[train_index:])
    
    y_train = y[:train_index]
    y_test = y[train_index:]
    return X_train, y_train, X_test, y_test

In [3]:
def train(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print('Accuracy:', f'{acc*100}%')
    print(confusion_matrix(y_test, y_pred))

In [10]:
X_train1, y_train1, X_test1, y_test1 = create_data(35, 0)
X_train2, y_train2, X_test2, y_test2 = create_data(35, 5)
X_train3, y_train3, X_test3, y_test3 = create_data(35, 10)

X_train = np.concatenate((X_train1, X_train2, X_train3), axis = 0)
y_train = np.concatenate((y_train1, y_train2, y_train3), axis = 0)
X_test = np.concatenate((X_test1, X_test2, X_test3), axis = 0)
y_test = np.concatenate((y_test1, y_test2, y_test3), axis = 0)

In [5]:
def pipeline(model, participants):
    X_train = ()
    y_train = ()
    X_test = ()
    y_test = ()
    for participant in participants:
        X_train1, y_train1, X_test1, y_test1 = create_data(participant, 0)
        X_train2, y_train2, X_test2, y_test2 = create_data(participant, 5)
        X_train3, y_train3, X_test3, y_test3 = create_data(participant, 10)
        
        X_train += (X_train1, X_train2, X_train3)
        y_train += (y_train1, y_train2, y_train3)
        X_test += (X_test1, X_test2, X_test3)
        y_test += (y_test1, y_test2, y_test3)
        

    X_train = np.concatenate(X_train, axis = 0)
    y_train = np.concatenate(y_train, axis = 0)
    X_test = np.concatenate(X_test, axis = 0)
    y_test = np.concatenate(y_test, axis = 0)
    
    train(model, X_train, y_train, X_test, y_test)
    return model

In [7]:
clf = SVC()
# train model in participant 31, 34 and 35
model = pipeline(clf, [31, 34, 35])

C:\Users\Admin\.pyenv\pyenv-win\versions\3.6.8\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Accuracy: 56.6131498470948%
[[ 517  737   90]
 [ 217 1429  298]
 [ 426  502 1016]]


In [ ]:
# train model in all data 
model = pipeline(clf, range(1, 61))

In [11]:
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.6075102880658436
